<a href="https://colab.research.google.com/github/gabrieladamasceno/Attacks_5G/blob/main/Aprendizado_por_Refor%C3%A7o_DDoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate
!pip uninstall gym -y
!pip install gymnasium
!pip install stable-baselines3

created virtual environment CPython3.10.12.final.0-64 in 558ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.2.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2


In [5]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import torch
from stable_baselines3 import DQN
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
# Carregamento do dataset
from google.colab import drive
drive.mount('/content/gdrive')

encoded = pd.read_csv("/content/gdrive/MyDrive/Datasets/Attacks/Encoded/Encoded.csv")

# Pré-processamento dos dados
features = [
    'Seq', 'Offset', 'sTtl', 'tcp', 'AckDat', 'RST', 'INT', 'TcpRtt', 'icmp',
    'sMeanPktSz', 'FIN', 'sHops', 'Mean', 'dTtl', 'SrcBytes', 'TotBytes',
    'dMeanPktSz', 'Status', 'cs0', 'SrcWin', 'sTos', 'CON', 'REQ', 'Loss'
]

# Features e os rótulos
X = encoded[features].values
y = encoded['Label'].values

# Pré-processar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Verificar e converter a coluna 'Label' (Benign/Malign) para valores numéricos
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
train_data = np.column_stack((X_train, y_train))
test_data = np.column_stack((X_test, y_test))




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
# Definição do ambiente de aprendizado por reforço
class AttackClassificationEnv(gym.Env):
    def __init__(self, data):
        super(AttackClassificationEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.action_space = spaces.Discrete(2)  # Ações 0 e 1
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.data.shape[1] - 1,), dtype=np.float32)  # Exclui o rótulo

    def reset(self, seed=None):
        self.current_step = 0
        if seed is not None:
            np.random.seed(seed)  # Define a semente se for passada
        return self.data[self.current_step, :-1].astype(np.float32), {}  # Retorna a observação inicial (sem o rótulo)

    def step(self, action):
        correct_label = self.data[self.current_step, -1]
        reward = 1 if action == correct_label else -1
        self.current_step += 1
        done = self.current_step >= len(self.data)

        if not done:
            next_state = self.data[self.current_step, :-1].astype(np.float32)
        else:
            next_state = np.zeros(self.observation_space.shape, dtype=np.float32)  # Estado final (pode ser None)

        # Compatibilidade com Stable Baselines3
        #next_state: próximo estado do ambiente
        #reward: recompensa recebida
        #done: indica se o episódio terminou
        #truncated: indica se a interação foi truncada
        #info: informações adicionais sobre a interação
        terminated = False  # Você pode definir como True se for o caso
        truncated = False  # Para simplicidade, podemos assumir que não há truncamento.

        return next_state, reward, done, truncated, {}  # Retorna 5 valores (por hora não tem nada no info)


# Criar o ambiente de treino
train_env = AttackClassificationEnv(train_data)

# Treinamento do agente DQN
model = DQN('MlpPolicy', train_env, verbose=1, exploration_fraction=0.3, exploration_final_eps=0.1, learning_rate=1e-4, buffer_size=50000)
model.learn(total_timesteps=50000)

# Avaliação do modelo
test_env = AttackClassificationEnv(test_data)
obs, _ = test_env.reset()  # Reset do ambiente de teste
done = False
correct_predictions = 0
total_predictions = 0

while not done:
    action = model.predict(obs)[0]
    obs, reward, done, truncated, info = test_env.step(action)
    total_predictions += 1
    if reward == 1:  # Recompensa 1 indica que a previsão estava correta
        correct_predictions += 1

# Calcular acurácia
accuracy = correct_predictions / total_predictions
print(f'Acurácia: {accuracy * 100:.2f}%')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Acurácia: 40.46%
